### Opakování &ndash; užitečné věci z Pythonu

In [4]:
my_string = "to be Or NOT to be"
my_list = my_string.split()
print 'Delka stringu je:', len(my_string)
print 'String obsahuje slova:', my_string.split()
print 'String obsahuje unikatni slova:', set(my_string.split())
print 'String prevedeny na mala pismena: ', my_string.lower()
print 'Delka pole (pocet slov) je:', len(my_list)
print 'Prvni prvek v poli je:', my_list[0]
print 'Posledni prvek v poli je:', my_list[-1]

Delka stringu je: 18
String obsahuje slova: ['to', 'be', 'Or', 'NOT', 'to', 'be']
String obsahuje unikatni slova: {'be', 'Or', 'NOT', 'to'}
String prevedeny na mala pismena:  to be or not to be
Delka pole (pocet slov) je: 6
Prvni prvek v poli je: to
Posledni prvek v poli je: be


### Spuštění interaktivního shellu PySpark

`pyspark --master yarn --num-executors 2 --executor-memory 4G --conf spark.ui.port=1<ddmm>`, kde `<ddmm>` je váš den a měsíc narození, např. `spark.ui.port=10811`

### Úkol č. 1: rozběhání příkladu z přednášky (word count)

Vyzkoušejte, že příklad z přednášky funguje.

In [ ]:
# nacteni RDD
lines = sc.textFile("/user/pascepet/data/bible/bible.txt")

# rozdeleni radku
words = lines.flatMap(lambda line: line.split(" "))

# transformace radku na (key, value)
pairs = words.map(lambda word: (word, 1))

# secteni jednicek ke kazdemu klici
counts = pairs.reduceByKey(lambda a, b: a + b)

# vypis vysledku
counts.take(5)

### Úkol č. 2: vylepšení příkladu z přednášky

Vylepšete skript z úkolu č. 1 o následující funkcionality:

2.1. Ignorujte začátky řádků (název biblické knihy a kód kapitola:verš &ndash; jsou odděleny od textu tabulátorem).  
2.2. Slova, která se liší jen velikostí písmen, se budou považovat za stejná.  
2.3. Odstraňte z textu nealfanumerické znaky &ndash; všechny nebo aspoň některé z nich (např. '.', ':', '-' atd.).  
2.4. Vyřaďte ze zpracování slova v množině tzv. *stopwords* (v souboru `/user/pascepet/data/stopwords.txt`).   
*Hint: stopwords můžete dostat do proměnné typu set např. takto:*  
`sw = set(sc.textFile('/user/pascepet/data/stopwords.txt').collect())`  
2.5. Vyberte RDD, které je vhodné si pro další výpočty kešovat, a nakešujte ho.  
2.6. Na konci seřaďte slova podle četnosti sestupně (použijte [metodu sortBy](https://spark.apache.org/docs/1.6.0/api/python/pyspark.html)).  
--> Slova mimo stopwords s nejvyšším výskytem: 'lord' 7830; 'god' 4443; 'said' 3999; 'will' 3836; 'man' 2613...  
2.7. Najděte nejdelší slovo (s největším počtem znaků).  
--> 'mahershalalhashbaz' (18); 'chushanrishathaim' (17); 'kibrothhattaavah' (16); 'covenantbreakers' (16); 'evilfavouredness' (16)...

### Úkol č. 3: další analýza textu

V této části budeme pracovat s původním textem.

3.1. Zjistěte počet slov v každém verši (1 verš = 1 řádek) a najděte verše s největšim a nejmenším počtem slov.  
--> 'Esther 8:9' 87 slov; 'John 11:35' 2 slova  
3.2. Proveďte stejný výpočet jako v 3.1, ale v každém verši počítejte jen unikátní slova (každé slovo jen jednou).  
--> 'Esther 8:9' 57 slov; 'John 11:35' 2 slova

### Úkol č. 4: číselné výpočty z dat

#### Data

Budeme pracovat se soubory hodinových údajů o teplotách (použili jsme ho na supercvičení Hive). Pokud jste na tomto supercvičení byli, pravděpodobně už máte data na HDFS (v podadresáři svého pracovního adresáře, jsou to soubory `teplota1-6.csv` a `teplota7-12.csv`) Zkontrolujte to.  
Nemáte-li je tam, proveďte následující kroky:

* zkopírujte soubor `/home/pascepet/fel_bigdata/data/teplota-usa.zip` do svého uživatelského adresáře (doporučuji si na to vytvořit podadresář);
* rozbalte;
* vytvořte na HDFS ve svém uživatelském adresáři vhodný podadresář;
* rozbalené soubory nakopírujte na HDFS do nově vytvořeného adresáře.

CSV soubor má jako oddělovač znak `','`. Také obsahuje hlavičky s názvy sloupců, které je potřeba při zpracování odstranit.    
Teplota je uvedena v 10&times;&deg;F. Některé řádky obsahují na místě teploty prázdný řetězec, tj. údaj pro ně není k dispozici.

**Sloupce:** id_stanice, mesic, den, hodina, teplota, flag, latitude, longitude, vyska, stat, nazev

*Poznámka: Spark umí v metodě textFile načíst celý adresář (spojí všechny soubory v adresáři do jednoho RDD).*

#### Zadání

4.1. Prohlédněte si několik prvních řádků z některého souboru, abyste si připomněli, co v datech je.  
4.2. Zjistěte, který stát ma nejvyšší průměrnou teplotu z měření jen za měsíce 6&ndash;8. Teplotu uveďte ve stupních Celsia.  
4.3. Pro každý měsíc vypište stát, který má nejvyšší průměrnou teplotu z měření za tento měsíc.


#### Očekávaný výstup

V zadání 4.2

| stat | prum_teplota |
|------|--------------:|
| MH   | 28.10121053814158 | 


V zadání 4.3

| mesic | stat | prum_teplota |
|-------|------|-------------:|
| 1     | AS | 28.1482513153823|  
| 2     | AS | 28.268547273982037|  
| 3     | AS | 28.305594514570675|  
| 4     | PW | 28.14697855750485|  
| 5     | PW | 28.15864931145062|  
| 6     | MH | 28.085874246339472|  
| 7     | AZ | 29.07906528274055|  
| 8     | TX | 28.335826966364785|  
| 9     | MH | 28.220327304048165|  
| 10     | MH | 28.22359756605796|  
| 11     | MH | 28.125538329026583|  
| 12     | AS | 27.996042413381048|  
